In [76]:
"""Test script that runs all the vehicles in both Excel and Python FASTSim for both UDDS and HWFET cycles."""

import xlwings as xw
import pandas as pd
import time
import numpy as np
import os
import sys
import importlib
import xlwings as xw
from math import isclose

from pathlib import Path
fsimpath = str(Path(os.getcwd()).parents[0])
if fsimpath not in sys.path:
    sys.path.append(fsimpath)

# local modules
from fastsim import simdrive, vehicle, cycle

In [115]:
def run_python_fastsim():
    """Runs python fastsim through 26 vehicles and returns list of dictionaries 
    containing scenario descriptions."""

    cycles = ['udds', 'hwfet']
    vehicles = np.arange(1, 27)
    print('Running python fastsim.')
    print('Instantiating classes.')
    print()
    veh = vehicle.Vehicle(1)
    veh_jit = veh.get_numba_veh()
    cyc = cycle.Cycle('udds')
    cyc_jit = cyc.get_numba_cyc()
    
    res_python = {}

    vehno = 9
    veh.load_veh(vehno)
    veh_jit = veh.get_numba_veh()

    print('vehno =', vehno)
    t0 = time.time()
    res_dict = {}
    for cycname in cycles:
        cyc.set_standard_cycle(cycname)
        cyc_jit = cyc.get_numba_cyc()
        sim_drive = simdrive.SimDriveJit(cyc_jit, veh_jit)
        sim_drive.sim_drive()

        res_dict['fe_' + cycname] = sim_drive.mpgge
        res_dict['kW_hr__mi_' + cycname] = sim_drive.electric_kWh_per_mi
    res_python[veh.Scenario_name] = res_dict

    t1 = time.time()
    print()
    print('Elapsed time: {:.2f} s'.format(t1 - t0))

    res_python['cpu_time'] = t1 - t0
        
    return res_python


def run_excel_fastsim():
    """Runs excel fastsim through 26 vehicles and returns list of dictionaries 
    containing scenario descriptions."""

    print('')
    print('Running excel fastsim.')
    # initial setup
    wb = xw.Book('FASTSim.xlsm')  # FASTSim.xlsm must be open
    sht_veh = wb.sheets('VehicleIO')
    sht_udds = wb.sheets('UDDS')
    sht_hwy = wb.sheets('HWY')
    sht_vehnames = wb.sheets('SavedVehs')
    app = wb.app
    load_veh_macro = app.macro("FASTSim.xlsm!reloadVehInfo")
    run_macro = app.macro("FASTSim.xlsm!run.run")

    vehicles = np.arange(1, 27)
    res_excel = {}
    
    vehno = 9
    print('vehno =', vehno)
    # running a particular vehicle and getting the result
    res_dict = {}
    sht_veh.range('C6').value = vehno
    load_veh_macro()
    t0 = time.time()
    run_macro()
    res_dict['fe_udds'] = sht_udds.range(
        'C118').value if sht_udds.range('C118').value != None else 0
    res_dict['fe_hwfet'] = sht_hwy.range(
        'C118').value if sht_hwy.range('C118').value != None else 0
    res_dict['kW_hr__mi_udds'] = sht_udds.range('C120').value
    res_dict['kW_hr__mi_hwfet'] = sht_hwy.range('C120').value

    res_excel[sht_vehnames.range('B' + str(vehno + 2)).value] = res_dict

    t1 = time.time()
    print()
    print('Elapsed time: {:.2f} s'.format(t1 - t0))
    res_excel['cpu_time'] = t1 - t0

    return res_excel


def compare(res_python, res_excel):
    """Finds common vehicle names in both excel and python 
    (hypothetically all of them, but there may be discrepancies) and then compares
    fuel economy results.  
    Arguments: results from run_python_fastsim and run_excel_fastsim
    Returns dict of comparsion results."""

    common_names = set(res_python.keys()) & set(res_excel.keys()) - {'cpu_time'}
    
    res_comps = {}
    for vehname in common_names:
        res_comp = {}
        if (res_excel[vehname]['kW_hr__mi_udds'] != 0) & (res_excel[vehname]['kW_hr__mi_hwfet'] != 0):
            res_comp['udds_elec_per_err'] = (
                res_python[vehname]['kW_hr__mi_udds'] -
                res_excel[vehname]['kW_hr__mi_udds']) / res_excel[vehname]['kW_hr__mi_udds'] * 100

            res_comp['hwfet_elec_per_err'] = (
                res_python[vehname]['kW_hr__mi_hwfet'] -
                res_excel[vehname]['kW_hr__mi_hwfet']) / res_excel[vehname]['kW_hr__mi_hwfet'] * 100
        else:
            res_comp['udds_elec_per_err'] = 0
            res_comp['hwfet_elec_per_err'] = 0 

        if (res_excel[vehname]['fe_udds'] != 0) & (res_excel[vehname]['fe_hwfet'] != 0):

            res_comp['udds_perc_err'] = (
                res_python[vehname]['fe_udds'] - res_excel[vehname]['fe_udds']) / res_excel[vehname]['fe_udds'] * 100

            res_comp['hwy_perc_err'] = (
                res_python[vehname]['fe_hwfet'] - res_excel[vehname]['fe_hwfet']) / res_excel[vehname]['fe_hwfet'] * 100
        else:
            res_comp['udds_perc_err'] = 0
            res_comp['hwy_perc_err'] = 0
            
        res_comps[vehname] = res_comp
        print('')
        print(vehname)
        
        print('FE % Error, UDDS: {:.2E}%'.format(res_comps[vehname]['udds_perc_err']))
        print('FE % Error, HWY: {:.2E}%'.format(res_comps[vehname]['hwy_perc_err']))
        print('kW-hr/mi % Error, UDDS: {:.2E}%'.format(res_comps[vehname]['udds_elec_per_err']))
        print('wK-hr/mi % Error, HWY: {:.2E}%'.format(res_comps[vehname]['hwfet_elec_per_err']))
        
    res_comps['py_speedup_factor'] = res_excel['cpu_time'] / res_python['cpu_time']
    print('Python gives a {:.2f}x speedup.'.format(res_comps['py_speedup_factor']))
    print("That is {:.2f} orders of magnitude faster.".format(np.log10(res_comps['py_speedup_factor'])))
    
    return res_comps

In [105]:
res_python = run_python_fastsim()

Running python fastsim.
Instantiating classes.

vehno = 9

Elapsed time: 0.03 s


In [106]:
res_excel = run_excel_fastsim()


Running excel fastsim.
vehno = 9

Elapsed time: 18.54 s


In [116]:
res_comp = compare(res_python, res_excel)


2016 FORD C-MAX HEV
FE % Error, UDDS: 5.90E-04%
FE % Error, HWY: 3.08E-08%
kW-hr/mi % Error, UDDS: 0.00E+00%
wK-hr/mi % Error, HWY: 0.00E+00%
Python gives a 714.65x speedup.
That is 2.85 orders of magnitude faster.
